# Topic Clustering Using LDA

The purpose of this tutorial is to give you a basic understanding of Latent Dirichlet Allocation (LDA) from http://ai.stanford.edu/~ang/papers/jair03-lda.pdf and use it to implement a simple downscaled topic clustering on the newsgroup dataset from scikit-learn.

Section 1 will give some background to the mechanics and theory behind LDA. Section 2 will then tackle the task of implementing LDA to infer topics in documents based on their content. This section will provide you with skeleton code already written in Python 3 using the numpy, scipy, and scikit-learn libraries. 

If you do not have jupyter notebook installed then you probably aren't reading this, but see http://jupyter.readthedocs.io/en/latest/install.html

If you do not have a python 3 kernel installed for jupyter notebook see https://ipython.readthedocs.io/en/latest/install/kernel_install.html or https://stackoverflow.com/questions/28831854/how-do-i-add-python3-kernel-to-jupyter-ipython

If you do not have some of the libraries installed for your python 3 kernel, use the "Kernel -> Conda packages" dropdown menu in Jupyter if you used anaconda for your python 3 kernel, if not use the normal pip install commands.

## 1: Theory

### Background and terminology

Since we will be working in the setting of text corpora, we should clarify some of the terminology used in this setting:
<ul>
<li>A word is the basic unit of discrete data, defined to be an item from a vocabulary indexed by {1, . . . ,V}. 
<li> A document is a sequence of <i>N</i> words denoted by <b>w</b>=(<i>w</i><sub>1</sub>,<i>w</i><sub>2</sub>, . . . ,<i>w</i><sub>N</sub>), where <i>w</i><sub>n</sub> is the <i>n</i>th word in the sequence.</li>
<li>A corpus is a collection of <i>M</i> documents denoted by $D$ ={<b>w</b><sub>1</sub>,<b>w</b><sub>2</sub>, . . . ,<b>w</b><sub>M</sub>}</li>
</ul>

It is important to note that LDA works in other domains besides text collections, but this is the setting in which we will use it.

LDA is a generative probabilistic model that is used for modeling collections of discrete data. In our application we will be using it to model text corpora, or more specifically news group e-mails. The purpose of the model is to give us compact representations of the data in these collections, allowing us to process large collections while still retaining sufficient information to be able to perform for example classification and relevance measures. 

There have been several solutions for this type of information retrieval problem, such as the tf-idf (term frequency - inverse document frequency) scheme by Salton and McGill, 1983. This approach produces a term-by-document matrix X whose columns contain the tf-idf values for each of the documents in the corpus. This representation however did not provide significantly shortened representation of the corpora, or represent the inter- or intra- document statistics in a intuitive way. A step forward from this was given by LSI (latent semantic indexing) where singular value decomposition was used on the matrix X to offer a more compact representation. The authors of the method also argued that since the LSI features are linnear combinations of the basic tf-idf features, they incorporate some linguistical notions such as synonomy and polysemy.
The first step to providing a generative model was the <i>probabilistic</i> LSI (pLSI), which uses mixture models to model each word in a document. The mixture components are the "topics" and represented as multinomial random variables, allowing different words in the document to be genereated by different topics. The compact representation for each document is then the list of numbers representing the mixing proportions for the fixed set of topics. The method however gives no generative probabilistic model for getting these numbers, causing the number of parameters in the model to grow linearly with the corpus size. Also, since there is no probabilistic model for the mixture components that represent a document, there is no clear way of assigning a probability to a document that is outside the training set.

Both LSI and pLSI use the "bag-of-words" approach which assumes exchangeability within the words of the document as well as the documents themselves, meaning their order is of no importance. A theorem due to de Finetti (1990) states that any collection of exchangeable random variables has a representation as a mixture distribution—in general an infinite mixture. This means we must consider mixture models that capture the exchangeability of both documents and words if we wish to achieve exchangeable representations for them. It is this line of thinking that leads to LDA.




### Theory Behind LDA

As mentioned earlier, LDA is a generative probabilistic model for a corpus. It can be seen as a hierarchical Bayesian model with three levels: each document in a corpus is modeled as a finite random mixture over a latent set of topics, and each of these topics are characterized by a distribution of words. A graphical model for LDA using plate notation can be seen below:
![title](imgs/LDAPlateGM.png)
From here we can see the three levels of the model. $\alpha$ and $\beta$ and corpus level parameters, $\theta$ is a document level parameter for the M documents in the corpus, and $z$ and $w$ are word level parameters for the N words in a document.

The generative process according to LDA for each document <b>w</b> is then:
<ol>
<li>Choose N ∼Poisson(ξ)</li>
<li>Choose $\theta$∼Dir($\alpha$)</li>
<li>For each of the N words w<sub>n</sub>:
<ol type="a">
    <li>Choose a topic z<sub>n</sub> ∼Multinomial($\theta$).</li>
    <li>Choose a word w<sub>n</sub> from p(w<sub>n</sub> |z<sub>n</sub>,$\beta$), a multinomial probability conditioned on the topic z<sub>n</sub>.</li>
    </ol></li>
</ol>

There are however some simplifications to these steps that we will utilize. First, we assume that the dimensionality of the Dirichlet distribution, and therefore the dimensionality for the topic variable $z$ is known and fixed, meaning we assume a fixed known number of topics, $k$. Furthermore, the probabilities for words ($w$) are parameterized by a $k \times V$ matrix $\beta$ which defines $p(w^j = 1| z^i = 1) = \beta_{i,j}$, that we will estimate later and keep fixed. We also note that $N$ is independant of the other data generating variables $\theta$ and <b>z</b> so we will ignore the Poisson assumption and set it to a known fixed value (the length of the document).  

#### Dirichlet Distribution in LDA

The probability distribution for a $k$-dimensional Dirichlet random variable $\theta$ is defined as follows: 

<b>Eq. 1:</b>
![Eq 1](imgs/LDAEq1.png "Eq 1")


where $\alpha$ is $k$-dimensional with all elements larger than 0 and $\Gamma(x)$ is the Gamma function. The Dirichlet distribution has some advantegous advantageous qualities; it is in the exponential family, has finite dimensional sufficient statistics, and is conjugate to the multinomial distribution. These properties help us in running variational inference for the parameters later.

We can now express the joint distribution of a topic mixture $\theta$, a set of $N$ topics <b>z</b>, and
a set of $N$ words <b>w</b> given the corpus level parameters $\alpha,\beta$ as:

<b>Eq. 2:</b>
![Eq. 2](imgs/LDAEq2.png)
where the probability $p(z_n |\theta)$ is simply $\theta_i$ for the unique $i$ such that $z^i_n=1$. We can then obtain the marginal distribution over a document by integrating over $\theta$ and summing over $z$:

<b>Eq. 3:</b>
![Eq. 3](imgs/LDAEq3.png)


#### Comparison to other Latent Variable Models
In order to get feeling for how LDA works and what highlights its strengths, it can be helpful to relate it to other related models:

  a) Unigram Model

  b) Mixture of Unigrams Model

  c) pLSI Model
  


We will begin by examing the absolute simplest model, the unigram model: 

![Eq. 3](imgs/UniGramMdl.png)

This method has no latent variables and instead states that each word in a document is independantly drawn from a single multinomial distribution as seen here:

![Eq. 3](imgs/UniGramEq.png)


A slighly more complex model is the mixture of unigrams:

![Eq. 3](imgs/MixUniGramMdl.png)

This model incorporates a discrete latent topic variable, $z$. Here, each document <b>w</b> is generated by first sampling the topic variable $z$, and then generating all words from a conditional probability on that choice:

![Eq. 3](imgs/MixUniGramEq.png)

This effectively limits the modeling of words in a document to only being representative of one topic. The LDA model on the other hand allows for documents to exhibit multiple topics with different mixtures.

Finally we have the pLSI model which we mentioned earlier. It was a relatively popular model around the time that LDA was proposed, and is the model with highest generative capabilities of these three mentioned. 

![Eq. 3](imgs/PISLMdl.png)

pLSI proposes that each word is conditionally independant a "document label", $d$, given an unobserved topic $z$:

![Eq. 3](imgs/PISLEq.png)

This proposal aims to soften the constraint of having each document modeled as being generated from only one topic, as it is in the mixture of unigrams approach. It does so by incorporating the probability, $p(z | d)$ for a certain document $d$ as the mixture of topics that document. A true generative model cannot be created for this mixture however; as d is only a dummy index to the documents pISL was trained with, meaning it is a multinomial random variable with the same amount of possible values as training documents. This leads to the method only learning the topic mixtures, $p(z | d)$, for documents it has already seen, so there is no natural way to assign probability to an unseen document with it.
Another problem is that to model $k$ topics with pLSI you need K multinomial distributions with vocabulary size $V$ and $M$ mixtures over the hidden topics $k$ for each training document, resulting in $kV + kM$ parameters. Not only does this not scale well but it is also prone to overfitting.

LDA however treats the topic mixture weights as a $k$-parameter hidden variable, meaning the amount of parameters does not scale linnearly with the number training documents, and the generative model can still be used even with unseen documents.

We can see these differences geometrically as well if we examine the distribution over words as a $V$-1 dimensional on a vocabulary of size $V$ with another $k$-1 dimensional simplex spanning $k$ topics. We can set $V$ and $k$ to 3 for simplicity (3 words gives a two-dimensional triangle):

![title](imgs/UnigramSampling.png)

How this distribution is spread out and how it uses the topics distribution differs among the methods. The mixture of unigrams method pics a random point on the word simplex that corresponds to one of the topic simplex vertices k, and draws all the words for a document from the distribution corresponding to that point. pLSI assumes that all words in training documents belong to a single randomly chosen topic. The topics are drawn from a document-specific distribution, meaning each document has a topic distribution that sits on the topic simplex. The training documents then give an empirical distribution (with the marked 'x's) over the topic simplex. LDA instead models that <b>each word</b> in a document is drawn from a randomly chosen topic that is sampled from a distribution governed by a random parameter. Since this parameter is sampled once per document, it gives a smooth probability distribution over the topic simplex (the circular topology markers). 

Now that we know how LDA compares with other methods, lets take a look at how to do inference in LDA:

### Inference and Parameter Estimation with LDA

The main inference problem we will be interested in solving is the posterior distribution of the latent variables given a document, which would allow us to infer the topics associated with the document. This is given by the following equation:

\begin{equation}
p( \theta, \mathbf{z} \mid \mathbf{w}, \alpha, \beta) = \frac{p( \theta, \mathbf{z}, \mathbf{w} \mid \alpha, \beta)}{p(\mathbf{w} \mid \alpha, \beta)} 
\end{equation}

However, to compute the normalizing denominator we would rewrite equation 3 using equation 1 and $p(z_n \mid\theta)=\theta_i$ and then integrate, resulting in:

\begin{equation}
p( \mathbf{w} \mid \alpha, \beta) = \frac{\Gamma(\sum_i\alpha_i)}{\prod_i\Gamma(\alpha_i)}\int\Bigg(\prod_{i=1}^k\theta_i^{\alpha_i-1}\Bigg)\Bigg(\prod_{n=1}^N\sum_{i=1}^k\prod_{j=1}^V(\theta_i\beta_{ij})^{w_n^j}\Bigg)d\theta
\end{equation}

Unfortunately, this expression is intractable due to the coupling of $\theta$ and $\beta$ in the summation over topics. We can however solve this problem approximately using variational inference methods. 

#### Variational Inference for LDA

It is possible to use several VI methods for LDA, including La Place approximation, variational approximation, and MCMC methods. In our case, we will be using the convexity-based variational approximation that was mentioned in  Olga's tutorial. From there we learned that in this VI we attempt to reformulate/simplify the original graphical model by removing some dependencies and introducing free variational parameters. This leads to a family of distributions dependant on these variational parameters which form a lower bound on the log likelyhood. We then aim to find the parameter values that give the tightest lower bound. 

In our case, the problematic dependancy is between $\theta$ and $\beta$ which is introduced by the edges between $\theta, \mathbf{z}$ and $\mathbf{w}$ (remember w is a 'collision' node and is observed). If we simplify our model by removing these edges along with the <b>w</b> node, and introduce two variational parameters $\gamma$ and $\phi$ which give a family of distributions over the remaining latent variables, we are left with the graphical model shown on the right in the figure below:

![LDA VI](imgs/LDAVIGM.png "GM for the VI used for our LDA")

This results in the following distribution over the latent variables:

\begin{equation}
p( \theta, \mathbf{z} \mid \gamma, \phi) = q(\theta\mid\gamma)\prod_{n=1}^Nq(z_n\mid\phi_n)
\end{equation}

where the new Dirichlet parameters $\gamma$ and the multinomial parameters $\phi$ are the free variational parameters. Now having simplified our graphical model, we need to find the optimal values for the variational parameters ($\gamma^*, \phi^*$). From Olga's tutorial we know that this is equivalent to finding the values which minimize the KL divergence between the simplified distribution and the true posterior distribution:

\begin{equation}
( \gamma^*, \phi^*) = \arg\!\min_{(\gamma,\phi)}D\big(q( \theta, \mathbf{z} \mid \gamma, \phi) \mid\mid p( \theta, \mathbf{z} \mid \mathbf{w},\alpha, \beta\big)
\end{equation}

We do this by setting the derivatives of the KL divergence to zero w.r.t $\gamma$ and $\phi$ we get the following update equations for the parameters:

\begin{equation}
\phi_{ni} \propto \beta_{iw_n}\mathrm{exp}\big\lbrace\mathrm{E}_q\big[log(\theta_i)\mid\gamma\big]\big\rbrace = \beta_{iw_n}\mathrm{exp}\big\lbrace\Psi(\gamma_i)\big\rbrace
\end{equation}

\begin{equation}
\gamma_i = \alpha_i + \sum_{n=1}^N\phi_{ni}
\end{equation}

where $\Psi$ is the digamma function (first derivative of log $\Gamma$). It is important to note that these update equations derived from the KL divergence are dependant on a certain choice of <b>w</b>. This means that the shown approximation for the variational parameters is only valid for one set of words, and must therefore be calculated for each document when we use them later on. 

We must also find a way of estimating the $\beta$ matrix, as it is used in the approximations for the variational parameters. The log likelihood of the data given $\beta$ and $\alpha$ is intractable as we saw at the end of the previous section. However, it is possible to implement a variational EM procedure that gives us an approximation of the best value for $\beta$ by first maximizing a lower bound for the optimal variational parameters $\gamma^*,\phi^*$, then maximizing the lower bound w.r.t $\beta$ with the previously acquired variational parameters. Essentially we will iterate the following steps until a sufficient level of convergence:
<ol>
<li>(E-step) Find the optimizing values of the variational parameters {$\gamma^∗
_d,\phi^∗_d : d\in D$},for each document as described earlier.</li>
<li>(M-step) Maximize the resulting lower bound on the log likelihood w.r.t $\beta$ using:
\begin{equation}
\beta_{ij}\propto\sum_{d=1}^M\sum_{n=1}^{N_d}\phi^*_{dni}w^j_{dn}
\end{equation}
as well as maximize the resulting lower bound on the log likelihood w.r.t $\alpha$ (this will be given to you).
</ol>

In laymans terms, what we are essentially doing in the E-step is finding out "How prevalent are topics in the document across its words?". In the M-step we then ask "How prevalent are specific words across topics?". By using the answer from one question as a starting point for the other, we iteratively gain the answer to both. 

<span style="color:red">For proof for the update equations, see appendix of http://ai.stanford.edu/~ang/papers/jair03-lda.pdf</span>. This appendix also includes the derivation of the Newton-Raphson based method for updating $\alpha$.  

We have now seen the basic intuition behind LDA, and gone through methods for running inference based on the LDA model. In the next section we will put this knowledge in to practice.


## 2. Implementation

The goal of this task is to use LDA to create topics newsgroup documents, and infer the topic that new documents would belong to. In this setting, our document corpus is the Newsgroup dataset from scikit-learn, and a document is a certain document/e-mail. 

### Dataset

First we will load the dataset we will use for training and testing. We will simplify the example from the original paper to only do clustering for 4 topics, and only use 250 documents with a vocabulary of 750 words. While this does have an effect on the accuracy and performance of the algorithm, it's more important for you to be able to run the code in a managable amount of time. The documents I have chosen come from 4 different categories; "Christian Religion", "Hockey", "Space" and "Cars". This means that we have slightly unrealistic prior knowledge by assuming the exact correct number of topics, but don't worry there are bonus assignments in the end where you can play around with the number of topics. I have already compiled and done some preprocessing on the documents, as well as built the vocabulary dataset as pickle files. Run the code in the cell below and double check that you get the output "found 200 training and 50 test documents, with a vocabulary of 750 words". Do not worry if you get a warning regarding the version of CountVectorizer which is used for handling the vocabulary of the dataset. 

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle as pickle

vectorizer = pickle.load(open("Data/vectorizerNews.p", "rb"))
trainDocs = pickle.load(open("Data/trainDocsNews.p", "rb"))
testDocs = pickle.load(open("Data/testDocsNews.p", "rb"))

#also load the original docs that aren't pre-processed for viewing later
origTrainDocs = pickle.load(open("Data/trainDocsNewsOrig.p", "rb"))
origTestDocs = pickle.load(open("Data/testDocsNewsOrig.p", "rb"))

print("Found ", len(trainDocs), "training and ", len(testDocs), " test documents, with a vocabulary of ", len(vectorizer.get_feature_names()), " words.")

Found  200 training and  50  test documents, with a vocabulary of  750  words.


C:\Users\Mangemange\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.18.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### Parameter estimation

We must now find the optimal values for the variational parameters, as well as the values for $\alpha$ and the $\beta$ matrix that were introduced in the variational inference section. In order to follow the instructions given in the VI section we will need to do some setup first, so run the following cells:

In [11]:
#Imports
import numpy as np
import scipy.special as special
import scipy.optimize 
import time

#diGamma func from scipy, use this in your code!
diGamma = special.digamma

#Function definitions for maximizing the VI parameters. This will later be completed by you.
def maxVIParam(phi, gamma, B, alpha, M, k, Wd, eta,verboseNum=-1):
    #print("in maxVIParam")
    for d in range(M):
        if(d==verboseNum):
            verbose=True
        else:
            verbose=False
            
        N = len(Wd[d])
        thisB = np.array([[B[i][Wd[d][n]] for n in range(N)] for i in range(k)])
        #Initialization of vars, as shown in E-step.
        phi[d] = np.ones((N,k))*1.0/k
        gamma[d] = np.ones(k)*(N/k) + alpha
        
        converged = False
        j = 0 
        iter=0
        
        while(not(converged)):
            #YOUR CODE FOR THE E-STEP HERE
            # UPDATE OF PHI:
            
            phi[d] = thisB.T*np.exp(diGamma(gamma[d][None,:]))
            normPhi = np.sum(phi[d],axis=(1)) # Sum over all (columns) i (topics) => Nx1
            
            if(np.min(normPhi)==0):
                print("trying to divide with 0: ")
                print(normPhi)
                problemIdx = np.array(range(len(normPhi)))[np.where(normPhi==0)]
                for i in problemIdx:
                    print(phi[d][i,:])
                
            
            phi[d] /= normPhi[:,None]

            # UPDATE OF GAMMA:
            
            oldGamma = gamma[d]
            gamma[d] = alpha + np.sum(phi[d],axis=(0)) # Sum over all (rows) n (words) => 1xk
            converged = (max(np.abs(oldGamma-gamma[d])) < eta)
            iter+=1
    return gamma, phi

#Function definitions for maximizing the B parameter. This will later be completed by you.
def MaxB(B, phi, k, V, M, Wd):
    
    #YOUR CODE FOR THE M-STEP HERE
    beta = np.zeros((k,V))

    for d in range(M):
        for w in range(len(Wd[d])):
            beta[:,Wd[d][w]] += phi[d][w,:]
    
    beta /= np.sum(beta,axis=(1))[:,None]
    return beta


In [12]:
'''Here are the functions needed for updating the alpha parameter, as shown in the start of appendix A.4.2.
These are all provided for you as it is just plugging in the definition for the gradient and hessian into the 
Newton-Raphson based method to find a stationary point using SciPy. Feel free to take a look at the appendix to
see where these values come from.'''

#value of Likelihood(gamma,phi,alpha,beta) function w.r.t. alpha terms (see start of appendix A.4.2) 
def L_alpha_val(a):
    val = 0
    M = len(gamma)
    k = len(a)
    for d in range(M):
        val += (np.log(scipy.special.gamma(np.sum(a))) - np.sum([np.log(scipy.special.gamma(a[i])) for i in range(k)]) + np.sum([((a[i] -1)*(diGamma(gamma[d][i]) - diGamma(np.sum(gamma[d])))) for i in range(k)]))

    return -val

#value of the derivative of above func w.r.t. alpha_i (2nd eq of appendix A.4.2) 
def L_alpha_der(a):
    M = len(gamma)
    k = len(a)
    der = np.array(
    [(M*(diGamma(np.sum(a)) - diGamma(a[i])) + np.sum([diGamma(gamma[d][i]) - diGamma(np.sum(gamma[d])) for d in range(M)])) for i in range(k)]
    )
    return -der

def L_alpha_hess(a):
    hess = np.zeros((len(a),len(a)))
    for i in range(len(a)):
        for j in range(len(a)):
            k_delta = 1 if i == j else 0
            hess[i,j] = k_delta*M*scipy.special.polygamma(1,a[i]) - scipy.special.polygamma(1,np.sum(a))
    return -hess

def MaxA(a):
    res = scipy.optimize.minimize(L_alpha_val, a, method='Newton-CG',
        jac=L_alpha_der, hess=L_alpha_hess,
        options={'xtol': 1e-8, 'disp': False})

    print(res.x)
    
    return res.x

With that in place, we can now initialize the required parameters and define the skeleton of our loop for the parameter estimation:

In [13]:
eta = 10e-5 #threshold for convergence

#hyperparamater init.
V = len(vectorizer.get_feature_names()) #vocab. cardinality
M = int(len(trainDocs)) #number of documents
k = 4 #amount of emotions

nIter=200 # number of iterations to run until parameter estimation is considered converged

#initialize B matrix as random valid distr (most common according to https://profs.sci.univr.it/~bicego/papers/2015_SIMBAD.pdf)
B = np.random.rand(k,V)

#normalize B
for i in range(k):
    B[i,:] = B[i]/np.sum(B[i])
    
alpha = np.ones(k)
#variational params (one for each doc)
phi = [None]*M
gamma = [None]*M

#word lists for all docs
Wd = [None]*M

'''Since scikit gives a matrix of counts of all words, and we want a list of words,
we do some quick transformations here. This gives us a representation of the documents 
as a list of numbers, where each number is the vocabulary index of a word. This way, to access
B_ij where i is the ith topic and j is the nth word in the document d, you can simply write B[i][Wd[d][n]]. If you want
replace this code a different representation for the words in a document, such as a one-hot vector for each word, you are
of course free to do so but make sure to keep track of your indexes'''

for d in range(M):
    Wmat = vectorizer.transform([trainDocs[d]]).toarray()[0] #get vocabulary matrix for document
    WVidxs = np.where(Wmat!=0)[0]
    WVcounts = Wmat[WVidxs]
    N = np.sum(WVcounts)
    W = np.zeros((N)).astype(int)

    i = 0
    for WVidx, WV in enumerate(WVidxs):
        for wordCount in range(WVcounts[WVidx]):
            W[i] = WV
            i+=1
    Wd[d] = W #We save the list of words for the document for analysis later

#[print(np.min(Wd[d])) for d in range(M)]
start = time.time()

#start of parameter estimation loop
for j in range(nIter):
    print("on iter: ", j)
    #Variational EM for gamma and phi (E-step from VI section)
    start = time.time()
    gamma, phi = maxVIParam(phi, gamma, B, alpha, M, k, Wd, eta)
    end = time.time()
    Bold = np.copy(B)
    B = MaxB(B,phi,k,V,M,Wd) #first half of M-step from VI section 
    # B re-normalized in MaxB
    
    print("B max diff: ", np.amax(abs(B-Bold)))
    end = time.time()
    
    alpha = MaxA(alpha) #second half of M-step from VI section 
    end = time.time()
    print("iter took: ", end-start)
    print("new alpha: ", alpha)
    
end = time.time()
print("took: ", end-start)

on iter:  0
B max diff:  0.022841456494436994
[1.74377368 1.64505421 1.67258949 1.43296063]
iter took:  5.092540740966797
new alpha:  [1.74377368 1.64505421 1.67258949 1.43296063]
on iter:  1
B max diff:  0.011264990630473336
[1.39694089 1.30137507 1.32298266 1.10519272]
iter took:  2.3868567943573
new alpha:  [1.39694089 1.30137507 1.32298266 1.10519272]
on iter:  2
B max diff:  0.008006439067811608
[0.87808522 0.8287542  0.82527307 0.6636437 ]
iter took:  1.8482789993286133
new alpha:  [0.87808522 0.8287542  0.82527307 0.6636437 ]
on iter:  3
B max diff:  0.0035607173597962463
[0.57665212 0.55139347 0.53144578 0.42366697]
iter took:  1.4438982009887695
new alpha:  [0.57665212 0.55139347 0.53144578 0.42366697]
on iter:  4
B max diff:  0.002671915542849963
[0.41748071 0.40087963 0.36313402 0.2950996 ]
iter took:  1.258800983428955
new alpha:  [0.41748071 0.40087963 0.36313402 0.2950996 ]
on iter:  5
B max diff:  0.002856420457206097
[0.31767505 0.30678614 0.25823063 0.21604141]
iter to

B max diff:  0.0003498440698841473
[0.09091766 0.07130969 0.06156733 0.0610254 ]
iter took:  0.7220289707183838
new alpha:  [0.09091766 0.07130969 0.06156733 0.0610254 ]
on iter:  46
B max diff:  0.0002859273387427223
[0.09036331 0.07095788 0.06164498 0.06064694]
iter took:  0.5696778297424316
new alpha:  [0.09036331 0.07095788 0.06164498 0.06064694]
on iter:  47
B max diff:  0.00023161251695472118
[0.0899985  0.07069073 0.06164269 0.06037017]
iter took:  2.39213228225708
new alpha:  [0.0899985  0.07069073 0.06164269 0.06037017]
on iter:  48
B max diff:  0.0001863534612935678
[0.08974675 0.07048638 0.06160406 0.06016564]
iter took:  0.7454507350921631
new alpha:  [0.08974675 0.07048638 0.06160406 0.06016564]
on iter:  49
B max diff:  0.0001487051812200978
[0.08956574 0.07032844 0.06155235 0.0600129 ]
iter took:  0.8479719161987305
new alpha:  [0.08956574 0.07032844 0.06155235 0.0600129 ]
on iter:  50
B max diff:  0.00012534565755585234
[0.08943109 0.07020422 0.06149919 0.05989767]
iter

[0.08238914 0.0691121  0.05809137 0.05694855]
iter took:  0.5050663948059082
new alpha:  [0.08238914 0.0691121  0.05809137 0.05694855]
on iter:  91
B max diff:  0.0004592893056751643
[0.08218771 0.06917009 0.05803558 0.05691335]
iter took:  0.453784704208374
new alpha:  [0.08218771 0.06917009 0.05803558 0.05691335]
on iter:  92
B max diff:  0.00046036806824496703
[0.08205395 0.0691925  0.05799783 0.05688173]
iter took:  0.48672938346862793
new alpha:  [0.08205395 0.0691925  0.05799783 0.05688173]
on iter:  93
B max diff:  0.0004186716260809898
[0.08195615 0.06919744 0.05797137 0.05685413]
iter took:  0.6296870708465576
new alpha:  [0.08195615 0.06919744 0.05797137 0.05685413]
on iter:  94
B max diff:  0.00035107771482371757
[0.08187741 0.0691937  0.05795165 0.05683018]
iter took:  0.554924726486206
new alpha:  [0.08187741 0.0691937  0.05795165 0.05683018]
on iter:  95
B max diff:  0.00027650120057396647
[0.08180802 0.06918509 0.05793546 0.05680919]
iter took:  0.6852519512176514
new al

B max diff:  0.0014328774108480283
[0.08004113 0.08197839 0.06140829 0.0612793 ]
iter took:  0.6015186309814453
new alpha:  [0.08004113 0.08197839 0.06140829 0.0612793 ]
on iter:  136
B max diff:  0.001124964369305221
[0.0807892  0.08267417 0.06167231 0.0616377 ]
iter took:  0.8550271987915039
new alpha:  [0.0807892  0.08267417 0.06167231 0.0616377 ]
on iter:  137
B max diff:  0.0009847439137363433
[0.08136805 0.08374741 0.06194904 0.06197351]
iter took:  0.77604079246521
new alpha:  [0.08136805 0.08374741 0.06194904 0.06197351]
on iter:  138
B max diff:  0.0007618419036896397
[0.08176427 0.0844498  0.06218754 0.06194058]
iter took:  0.6444129943847656
new alpha:  [0.08176427 0.0844498  0.06218754 0.06194058]
on iter:  139
B max diff:  0.0008390557080388689
[0.0820824  0.08544097 0.06241667 0.0620024 ]
iter took:  1.2197659015655518
new alpha:  [0.0820824  0.08544097 0.06241667 0.0620024 ]
on iter:  140
B max diff:  0.0005973958108271907
[0.08232784 0.08607804 0.06261852 0.06210071]
it

B max diff:  6.477312286980305e-05
[0.07963466 0.0892336  0.06519255 0.06159003]
iter took:  0.47353124618530273
new alpha:  [0.07963466 0.0892336  0.06519255 0.06159003]
on iter:  181
B max diff:  0.0001441334657975189
[0.07964497 0.08924239 0.06520286 0.06159549]
iter took:  0.4779949188232422
new alpha:  [0.07964497 0.08924239 0.06520286 0.06159549]
on iter:  182
B max diff:  0.0004917486871774219
[0.07965156 0.08924597 0.06521042 0.06159473]
iter took:  0.5156247615814209
new alpha:  [0.07965156 0.08924597 0.06521042 0.06159473]
on iter:  183
B max diff:  0.0013752934006374701
[0.07965366 0.08924315 0.06521503 0.06157969]
iter took:  0.4685788154602051
new alpha:  [0.07965366 0.08924315 0.06521503 0.06157969]
on iter:  184
B max diff:  0.0012756311518977907
[0.07964561 0.08923402 0.06521667 0.06155674]
iter took:  0.6874792575836182
new alpha:  [0.07964561 0.08923402 0.06521667 0.06155674]
on iter:  185
B max diff:  0.0008686192641278718
[0.07962478 0.08921934 0.06521542 0.06153573

#### VI Parameter Estimation
We can now begin with implementing the "E step" in the previous section which updates the variational parameters. The pseudo code for this is the following (remember these have to be calculated separately for each document):
![VIPseudo](imgs/VIPseudo.png)

Since we are working with four topics, k will be set to 4 and N will be the amount of words in the current document. Regarding the "until convergence" condition, it is sufficient to check if the largest difference between the previous and new gamma is less than $10^{-5}$. Now, use the pseudo code to fill in the missing code in the "MaxVIParam" function defined earlier and remember to use the provided diGamma function. To see that your implementation seems to be working, set nIter to 1 and add some printouts of the difference between updates for gamma, then check that they are converging to something smaller.

#### Beta Matrix Estimation
After you have implemented the MaxVIParam function, it's time to update the Beta matrix. Recall that the update function for Beta was:
\begin{equation}
\beta_{ij}\propto\sum_{d=1}^M\sum_{n=1}^{N_d}\phi^*_{dni}w^j_{dn}
\end{equation}
Implement this in the definition for MaxB above. To verify your code, you may set nIter to something low such as 10 and uncomment the "oldB" and "B max diff" lines in the code. The diff might increase at first but should start decreasing at least before iteration 10. After you have verified this, set nIter to 100 (updates should be negligable by then) and let it run unattended as it might take a couple hours. You can use the code in the following cell to save/load the parameter values you calculated for later so you don't have to re-run everything.

In [14]:
pickle.dump(alpha, open("Data/myAlphaNews100.p", "wb"))
pickle.dump(B, open("Data/myBetaNews100.p", "wb"))

#alpha = pickle.load(open("Data/myAlphaNews100.p", "rb"))
#B = pickle.load(open("Data/myBetaNews100.p", "rb"))

#### Verification
Let's take a look at what we've done so far. We can get an idea of what our implementation has done up to this point by inspecting the B matrix. As you may remember, B$_{ij}$ holds the probability of a vocabulary word j being representative of a certain topic i. Using the code in the following cell we can see the most representative words for our 4 topics:

In [15]:
#representation of top words for each topic:
nTop = 20
for i in range(k):
    topVocabs = np.argsort(B[i])[-nTop:][::-1]
    topWords = np.array(vectorizer.get_feature_names())[topVocabs]
    print("top words for topic ",i,": ")
    print(topWords)

top words for topic  0 : 
['team' 'flyer' 'play' 'game' 'hockey' 'ca' 'gm' 'year' 'season' 'player'
 'point' 'post' 'goal' 'good' 'leaf' 'city' 'nntp' 'think' 'win' 'time']
top words for topic  1 : 
['car' 'hell' 'say' 'think' 'father' 'know' 'time' 'son' 'spirit' 'thing'
 'believe' 'like' 'really' 'eternal' 'way' 'die' 'good' 'use' 'problem'
 'heaven']
top words for topic  2 : 
['god' 'people' 'jesus' 'say' 'christian' 'know' 'think' 'religion'
 'question' 'believe' 'life' 'world' 'law' 'truth' 'come' 'faith' 'church'
 'time' 'christ' 'paul']
top words for topic  3 : 
['space' 'nasa' 'year' 'launch' 'satellite' 'use' 'mission' 'project'
 'gov' 'post' 'data' 'access' 'orbit' 'news' 'earth' 'list' 'research'
 'probe' 'world' 'program']


Since there are no guarantees regarding the order of the topics (LDA is unsupervised) or what your initial B matrix values were, it is difficult to say exactly what results you should be seeing. Hopefully, you can see the four original topics to some extent in your result. For example, one of my topics had top words like "christ", and "god", meaning it was most likely the topic for "Christian Religion" documents, while another literally had the words "Hockey" and "NHL" in it. Our vocabulary is as mentioned earlier quite limited, so it may be possible that one of your topics is a bit unclear or close to another. You can also load the $\alpha$ and $\beta$ values in the cell below which are pre-calculated for 200 iterations and compare your topic results to those available there.

In [16]:
alphaTest = pickle.load(open("Data/CompareAlphaNews200.p", "rb"))
BTest = pickle.load(open("Data/CompareBetaNews200.p", "rb"))
vecTest = pickle.load(open("Data/vectorizerNews.p", "rb"), encoding='latin1')
nTop = 20
for i in range(k):
    topVocabs = np.argsort(BTest[i])[-nTop:][::-1]
    topWords = np.array(vecTest.get_feature_names())[topVocabs]
    print("top words for topic ",i,": ")
    print(topWords)

top words for topic  0 : 
['space' 'use' 'post' 'nasa' 'nntp' 'program' 'high' '1993' 'distribution'
 'year' 'science' 'new' 'question' 'world' 'national' 'development'
 'spacecraft' 'launch' 'satellite' 'data']
top words for topic  1 : 
['god' 'people' 'say' 'know' 'christian' 'believe' 'time' 'think' 'good'
 'question' 'thing' 'christ' 'way' 'come' 'use' 'make' 'like' 'word'
 'bible' 'reason']
top words for topic  2 : 
['team' 'play' 'year' 'player' 'hockey' 'game' 'season' 'nhl' 'contact'
 'wing' 'playoff' 'red' '86' 'star' '90' '93' '1992' '92' 'point' 'blue']
top words for topic  3 : 
['think' 'post' 'know' 'car' 'time' 'say' 'like' 'nntp' 'good' 'people'
 'use' 'come' 'year' 'way' 'thing' 'really' 'look' 'make' 'work' 'usa']


C:\Users\Mangemange\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.18.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


#### Inferring the topic of a test document

In this section we will be using our estimated parameter values to infer the topic of some test documents. In order to do this, we will have to calculate the phi and gamma values for each new document we would like to do inference on. This is rather straight forward, and you should be able to reuse your code from the previous sections together with the test documents as a corpus instead.

In [22]:
eta = 10e-5 #threshold for convergence

#we are not re-initializing beta and alpha, we calculated them using the training docs.

V = len(vectorizer.get_feature_names()) #vocab. cardinality
M = int(len(testDocs)) #number of documents
k = 4 #amount of emotions

#variational params (one for each doc)
phi = [None]*M
gamma = [None]*M
WdTest = [None]*M

'''Same magic from before to get the word matrix correct, replace this if you redid this earlier.'''
for d in range(M):
    Wmat = vectorizer.transform([testDocs[d]]).toarray()[0] #get vocabulary matrix for document
    WVidxs = np.where(Wmat!=0)[0]
    WVcounts = Wmat[WVidxs]
    N = np.sum(WVcounts)
    W = np.zeros((N)).astype(int)

    i = 0
    for WVidx, WV in enumerate(WVidxs):
        for wordCount in range(WVcounts[WVidx]):
            W[i] = WV
            i+=1
    WdTest[d] = W #We save the list of words for the document for analysis later

'''Now that you have your variables initialized for the test documents, you should be able to use your previous code for 
maximizing the VI parameters with those variables instead. Remember, we're just calculating the variational parameters
gamma and phi for each test document so there is no iteration between maximizing Beta and maximizing gamma and phi.'''

# YOUR CODE to Run the gamma/phi maximization here.

gamma, phi = maxVIParam(phi, gamma, B, alpha, M, k, WdTest, eta)


We have now calculated the variational parameters for our test documents, so let us see what information we can infer from them. If you take a look at the pseudo code we used for the MaxVIParam method, you can see that the posterior gamma parameter $\gamma_i $we are calculating is approximately the prior Dichlet parameter $\alpha_i$ added to the expected number of words that were generated by that $i^{th}$ topic for a certain document. Looking at the values for the different $\gamma_i$ over all words for a test document tells us what mixture of topics form such a document. Let us now take a look at the mixtures for some of our test documents by running the code in the cell below: 

In [24]:
#take a look at some example test documents (14-24 has a nice mix of topics, with a couple difficult ones)
dStart = 33
dEnd = 340
for d in range(dStart,dEnd):
    print("Estimated mixture for document ", d," is: ")
    print("_______________________")
    for i in range(len(gamma[d])):
        print("topic ", i,": ", gamma[d][i]/np.sum(gamma[d]))
    print("_______________________")
    print("Which has the following text:")
    print(" ")
    print(origTestDocs[d])
    #print(trainDocs[d])
    print("__________________________________________")
    print("__________________________________________")


Estimated mixture for document  33  is: 
_______________________
topic  0 :  0.8587046557860883
topic  1 :  0.0004639789395019427
topic  2 :  0.14052308649791462
topic  3 :  0.0003082787764950367
_______________________
Which has the following text:
 
From: nlu@Xenon.Stanford.EDU (Nelson Lu)
Subject: SHARKS REVIEW Part 3: Defensemen (21-45)
Organization: Computer Science Department, Stanford University.
Lines: 85

#21	PETER AHOLA		Season: 2nd
Acquired:	'92-93, trade with Pittsburgh for future considerations
Grade:		I (B)

It is way too early to tell about Ahola, who was acquired probably because the
Penguins figured that they would lose him in the expansion draft.  Ahola had
only played 50 games this season (I think it's actually less; the San Jose
Mercury News may be in err here), 20 of them with the Sharks.  In the games he
has played, he appeared quite solid defensively, although he hasn't been
spectacular, and his offense isn't anything to write home about (8 points);
it's even pos

IndexError: list index out of range

Revisit the cell that presented the top words for your topics. Do the presented mixtures above make sense if you look at the document content? Recall which original categories ("Religion", "Cars", "Hockey", "Space") you (to your best ability) assigned to which numbers. Do the texts seem to be discussing those topics?

<span style="color:blue">If you're re-doing this test with the MoodyLyrics dataset from the bonus section, you may be noticing some weird results. LDA can experience some issues in this setting, as for example many words that would be present in a happy song could also be present in a sad song ('love', 'hold', 'forever') but in different order or with certain "negating" words between them. It is possible to alleviate this problem by using a vocabulary of n-grams, however this increases the total size of the vocabulary (and therefore the run time as well) substantially. </span>

It is also possible to gain some more insight by examining the $\phi$ values for the documents. Recall that the $\phi$ values for the document approximate $p(z_n | \mathbf{w})$, showing how the topics are mixed for the words in the document. The cell below provides a method for printing the phi values for each word in a document. Apart from just examining how the topics are mixed for specific words, take a look at the topic mixtures for the same word that appears in several different documents. As stated in the theory section, in LDA the distribution of topic mixtures that are assigned to each word is sampled differently for each document. This means that hopefully it should be apparent from your results how the topic mixture for the same word can be differ in different test documents.

In [25]:
#14-24 gives a good mix, but try whatever you like
dStart = 0
dEnd = 10


def getWordsFromMatrix(WdTest):
    originalWords  = np.array(vectorizer.get_feature_names())[WdTest] 
    return originalWords

for dk in range(dStart,dEnd):
    
    origWords = getWordsFromMatrix(WdTest[dk])
    wordMixtures = [origWords[n] + "\t: " + str(phi[dk][n]) for n in range(len(phi[dk]))]
    for wm in set(wordMixtures):
        print(wm)
    print("________________________________")


eternal	: [0.00000000e+00 1.00000000e+00 1.04154322e-81 0.00000000e+00]
yes	: [6.31406106e-009 3.10193409e-001 6.89806585e-001 1.25388445e-213]
16	: [2.60426703e-08 1.01691476e-01 8.98308497e-01 2.21982931e-10]
come	: [9.06819888e-09 8.78717863e-02 9.12128205e-01 6.65115606e-11]
problem	: [8.65180835e-09 5.25422855e-01 4.74577136e-01 7.70178711e-27]
important	: [1.21240881e-08 2.78114751e-01 7.21885237e-01 7.56278445e-54]
talk	: [5.44592467e-21 1.81470932e-01 8.18529068e-01 2.70324263e-62]
lead	: [3.86754724e-08 8.89783644e-02 9.11021597e-01 3.60050543e-10]
answer	: [2.07796893e-165 2.49533791e-048 1.00000000e+000 1.78873467e-010]
accord	: [6.22080568e-25 1.84501932e-01 8.15498068e-01 1.61822877e-10]
christian	: [0.         0.02397333 0.97602667 0.        ]
rick	: [1.01341432e-07 0.00000000e+00 9.99999899e-01 0.00000000e+00]
uk	: [0.00000000e+000 1.00000000e+000 1.08952544e-020 4.14959549e-151]
help	: [6.68498733e-09 1.63908268e-01 8.36091725e-01 1.84629374e-10]
life	: [3.84428786e-93 

salvation	: [0.00000000e+00 3.63081904e-20 1.00000000e+00 0.00000000e+00]
scripture	: [0.         0.04123028 0.95876972 0.        ]
32	: [2.87718720e-08 1.10217720e-29 9.99999971e-01 3.98614825e-10]
reason	: [1.27337423e-09 3.76976208e-01 6.23023790e-01 1.00697229e-10]
ad	: [0.00000000e+000 1.00000000e+000 7.63927442e-221 1.20200078e-010]
department	: [4.09143246e-008 1.23744550e-137 9.99999958e-001 1.00015219e-009]
hold	: [3.75800069e-08 7.02833699e-01 2.97166263e-01 6.81834721e-10]
answer	: [2.31692437e-165 5.33486820e-048 1.00000000e+000 1.99460928e-010]
reserve	: [7.47825857e-08 9.11141766e-04 9.99088780e-01 3.15355267e-09]
need	: [1.04224905e-08 1.85095454e-01 8.14904536e-01 1.38515691e-10]
usa	: [2.42708490e-08 9.99999975e-01 3.73353467e-59 4.86260985e-10]
truth	: [2.22859682e-71 8.49647137e-02 9.15035286e-01 0.00000000e+00]
miss	: [1.69801342e-08 7.83812238e-56 9.99999982e-01 5.47555745e-10]
christianity	: [0.00000000e+000 3.23622922e-002 9.67637708e-001 1.91694254e-206]
rutgers

20	: [9.89105389e-01 5.58291997e-82 1.46642544e-09 1.08946094e-02]
14	: [9.94053340e-01 4.41556871e-09 1.35332930e-09 5.94665392e-03]
29	: [9.87033344e-01 6.62436252e-08 3.78073299e-11 1.29665897e-02]
result	: [5.63275858e-86 1.83123292e-19 1.40814171e-07 9.99999859e-01]
31	: [9.97734558e-01 3.12201899e-08 8.02883830e-10 2.26540957e-03]
pt	: [9.99999999e-01 0.00000000e+00 9.04049705e-10 0.00000000e+00]
________________________________
express	: [1.38283343e-01 2.80163698e-07 9.22004818e-02 7.69515895e-01]
sci	: [3.03525034e-184 2.82496631e-212 9.31548608e-003 9.90684514e-001]
pat	: [1.57431900e-01 2.14799301e-37 9.31089045e-03 8.33257210e-01]
space	: [3.86551157e-238 7.16027042e-122 1.63942958e-056 1.00000000e+000]
maybe	: [7.66093043e-01 6.36703932e-06 2.33900589e-01 1.25543319e-09]
digex	: [5.21723405e-145 5.59210102e-298 4.50065509e-233 1.00000000e+000]
usa	: [3.78793022e-01 1.44506134e-06 4.11281647e-60 6.21205533e-01]
actually	: [1.21081214e-01 8.91780787e-07 2.83192709e-01 5.9572

#### Bonus Objectives

Well done! You have now implemented LDA, approximated the necessary variational parameters, and examined the results to infer information about topics in documents. If you feel like you would like to experiment some more, there are some variants that you could try:

1. Load the provided dataset from the Associated Press docs dataset. This has random news articles from an undisclosed number of topics. Replace the dataset code in the beginning with what is provided in the next cell and redo your tests. What kind of topics does your result have? How many topics did you assume there were? (Some interesting cases I got were general topics like Crime and Economics and then one focusing solely on foreign affairs with President Bush)

2. Run the tests using the MoodyLyrics dataset instead. This dataset includes the lyrics from songs in many different genres (I've included has slightly less than 200 / 50 documents and V=500). Run the tests again and see what kind of sense LDA tries to make out of these song lyrics. The dataset also provides an annotation as to what emotion ("Angry", "Sad", "Happy", "Relaxed") the song exhibits. Can you find a resemblence in your topics to these emotions? (<i>Disclaimer: The lyrics provided are not censored and some are not exactly "PG-13")</i>

<b>It is possible to start to see results after ~50-60 iterations so if you would like to try out these bonus exercises you need not wait overnight</b>

In [10]:
#loading the AP docs dataset instead:
#(everything else should work like before)
'''vectorizer = pickle.load(open("Data/vectorizerAP.p", "rb"), encoding='latin1')
trainDocs = pickle.load(open("Data/trainDocsAP.p", "rb"), encoding='latin1')
testDocs = pickle.load(open("Data/testDocsAP.p", "rb"), encoding='latin1')

test1TrainDocs = pickle.load(open("Data/trainDocsAP.p", "rb"), encoding='latin1')
test1TestDocs = pickle.load(open("Data/testDocsAP.p", "rb"), encoding='latin1')

#loading the moodyLyrics dataset instead:
vectorizer = pickle.load(open("Data/vectorizerMoodyLyrics.p", "rb"), encoding='latin1')
trainLyricsFile = pickle.load(open("Data/trainDocsMoodyLyrics.p", "rb"), encoding='latin1')
testLyricsFile = pickle.load(open("Data/testDocsMoodyLyrics.p", "rb"), encoding='latin1')

trainDocs = trainLyricsFile['lyrics']
testDocs = testLyricsFile['lyrics']
trainGT = trainLyricsFile['groundTruth']
#original moods can be seen with: trainGT = trainLyricsFile['groundTruth'] but the labeling is not perfect. 
'''